In [ ]:
threads = 16
import os
os.environ['OMP_NUM_THREADS']=str(threads)
import tensorflow as tf

# Tensorflow needs explicit cofig calls
tf.config.threading.set_inter_op_parallelism_threads(threads)
tf.config.threading.set_intra_op_parallelism_threads(threads)

In [ ]:
# input conformation
#conf = "alaninedipeptide_H.pdb"
conf = "trpcage_correct.pdb"

# input trajectory
# atom numbering must be consistent with {conf}

#traj = "alaninedipeptide_reduced.xtc"
traj = "trpcage_red.xtc"

# input topology
# expected to be produced with 
#    gmx pdb2gmx -f {conf} -p {topol} -n {index} -o {gro}

# Gromacs changes atom numbering, the index file must be generated and used as well
# gro file is used to generate inverse indexing for plumed.dat

#topol = "topol.top"
topol = "topol_correct.top"
index = 'index_correct.ndx'
gro = 'trpcage_correct.gro'

In [ ]:
import mdtraj as md
import numpy as np
import asmsa
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

In [ ]:
tr = md.load(traj,top=conf)
idx=tr[0].top.select("name CA")
#idx=tr[0].top.select("element != H")
tr.superpose(tr[0],atom_indices=idx)
geom = np.moveaxis(tr.xyz ,0,-1)
geom = np.moveaxis(tr.xyz ,0,-1)
geom.shape

In [ ]:
density = 2 # integer in [1, n_atoms-1]

sparse_dists = asmsa.NBDistancesSparse(geom.shape[0], density=density)
mol = asmsa.Molecule(pdb=conf,top=topol,ndx=index,fms=[sparse_dists])

In [ ]:
X_train = mol.intcoord(geom).T
X_train.shape

In [ ]:
ds = tf.data.Dataset.from_tensor_slices(X_train).shuffle(2048).batch(256,drop_remainder=True)

In [ ]:
en = list(range(32,129,32))
dn = list(range(64,257,64))
el = len(en)
dl = len(dn)
en = en * dl
dn = list(np.repeat(np.array(dn),el,axis=0))

In [ ]:
list(en)

In [ ]:
dn

In [ ]:
#m = asmsa.AAEModel((X_train.shape[1],),enc_seed=[32,32,64,64,128,128],disc_seed=[64,128,64,128,64,128])
m = asmsa.AAEModel((X_train.shape[1],),enc_layers=3,disc_layers=3,enc_seed=en,disc_seed=dn)
m.compile(optimizer='Adagrad')

In [ ]:
m.fit(ds,epochs=10)

In [ ]:
lows = m.enc(X_train[::500]).numpy()
#lows = m5.aes(X_train[::500])[1].numpy()

plt.figure(figsize=(12,9))
for mod in range(lows.shape[1]//2):
    plows = lows[:,mod*2:(mod+1)*2]
    plt.scatter(plows[:,0],plows[:,1],marker='.',label=str(mod))

lim=2
plt.legend()    
plt.xlim((-lim,lim))
plt.ylim((-lim,lim))
plt.show()

In [ ]:
keras.optimizers.__dict__['Adam']